In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import tkinter as tk
from tkinter import filedialog
import os
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper
from bokeh.transform import linear_cmap
from bokeh.layouts import column
from bokeh.palettes import Viridis256

output_notebook()  # for Jupyter; remove if using standalone script

def load_ddc_data():
    """
    Prompts the user to select a .mat or .npz file via GUI dialog,
    and loads localization and frame data.

    Returns:
    - Frame_Information
    - LocalizationsFinal
    - file_path (str): Path to the selected file
    """
    # Use tkinter to open file dialog
    root = tk.Tk()
    root.withdraw()  # Hide the main tkinter window

    file_path = filedialog.askopenfilename(
        title="Select DDC Data File (.mat or .npz)",
        filetypes=[("MAT files", "*.mat"), ("NPZ files", "*.npz")]
    )

    if not file_path:
        raise FileNotFoundError("No file was selected.")

    ext = os.path.splitext(file_path)[1].lower()

    if ext == '.mat':
        data = loadmat(file_path, verify_compressed_data_integrity=False, struct_as_record=False, squeeze_me=True)
        FI = data['Frame_Information']
        LF = data['LocalizationsFinal']
    elif ext == '.npz':
        npz = np.load(file_path, allow_pickle=True)
        FI = npz['Frame_Information'].tolist()
        LF = npz['LocalizationsFinal'].tolist()
    else:
        raise ValueError("Unsupported file format. Please provide a .mat or .npz file.")

    return FI, LF


def visualize_storm_localizations(xy_coords, frame_info=None, color_by='frame'):
    """
    Visualize 2D STORM localization data using Bokeh.

    Parameters:
    - xy_coords: Nx2 numpy array or list of (x, y) coordinates
    - frame_info: N-length list/array of frame numbers (optional)
    - color_by: 'frame' or 'none'
    """

    coords = np.asarray(xy_coords)
    x, y = coords[:, 0], coords[:, 1]

    source_data = {'x': x, 'y': y}
    
    if color_by == 'frame' and frame_info is not None:
        source_data['frame'] = np.asarray(frame_info)
        color_mapper = linear_cmap(field_name='frame', palette=Viridis256,
                                   low=min(frame_info), high=max(frame_info))
    else:
        color_mapper = 'black'

    source = ColumnDataSource(data=source_data)

    p = figure(
        title="2D STORM Localization Plot",
        x_axis_label='X (nm)', y_axis_label='Y (nm)',
        width=700, height=700,
        match_aspect=True,
        tools='pan,wheel_zoom,box_zoom,reset,save'
    )

    if color_by == 'frame' and frame_info is not None:
        r = p.circle('x', 'y', source=source, size=2, color=color_mapper, alpha=0.6)
        color_bar = ColorBar(color_mapper=color_mapper['transform'], label_standoff=12, location=(0,0))
        p.add_layout(color_bar, 'right')
    else:
        p.circle('x', 'y', source=source, size=2, color=color_mapper, alpha=0.6)

    show(p)


Loading BokehJS ...

In [4]:
Frame, loc = load_ddc_data()

In [7]:
visualize_storm_localizations(loc)